<a href="https://colab.research.google.com/github/kbrezinski/CS224W-GraphML/blob/main/notebooks/practice-GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.__version__)

from platform import python_version
print(python_version())

1.11.0+cu113
3.7.13


In [ ]:
!pip install torch-scatter torch-sparse \
 torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-1.11.0+cu113.html -q
#!pip install ogb

     |████████████████████████████████| 7.9 MB 6.5 MB/s 
     |████████████████████████████████| 3.5 MB 47.6 MB/s 
     |████████████████████████████████| 2.5 MB 2.7 MB/s 
     |████████████████████████████████| 750 kB 45.6 MB/s 
     |████████████████████████████████| 407 kB 7.9 MB/s 


In [ ]:
import os
import numpy as np

from torch_geometric.data import Dataset, Data, Batch
from torch_geometric.loader import ClusterData, NeighborLoader, DataLoader

In [ ]:
# torch_geometric.data.batch.Batch 
# -creates batch of disconencted graphs from list

# torch_geometric.data.data.Data
# -creates single graph object

# torch_geometric.data.cluster.ClusterData/ClusterLoader
# -group nodes into smaller subgraphs and load them in batches for faster computation

# torch_geometric.data.sampler.NeighborSampler
# -samples specific number of nodes in neighbor
# -sample training nodes only using node_idx

In [ ]:
import pickle

rows = np.random.choice(100, 500)
cols = np.random.choice(100, 500)

data = dict(x = torch.rand((100, 16), dtype=torch.float), # 100 nodes, 16 features)
            edge_index = torch.tensor([rows, cols]), # (2, 500) random edges
            edge_attr = np.random.choice(3, 500), # 500 edges, choose from 0, 1 or 2
            y = torch.rand(100).round().long(),
)  

g = Data(**data)

In [ ]:
from torch_geometric.nn import MessagePassing

class GCNConv(MessagePassing):
  def __init__(self, in_channels, out_channels):
    super().__init__(aggr='add')
    self.lin = torch.nn.Linear(in_channels, out_channels)

  def forward(self, x, edge_index):
    # x = (N, in_channels)
    # edge_index = (2, E)

    # 1. Add self loops
    edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
    # 2. Linear tranformation
    x = self.lin(x)
    # 3. Normalization
    row, col = edge_index
    deg = degree(col, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    # 4-5. Propogate messages
    return self.propogate(edge_index, x=x, norm=norm)

  def message(self, x_j, norm):
    return norm.view(-1, 1) * x_j
